In [ ]:
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import sklearn
import torch
import torch.nn as nn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed as TD
from tensorflow.keras.layers import LSTM, BatchNormalization, SeparableConv2D, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16, ResNet50, VGG19
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 이미지의 가로 세로
IMG_SIZE = 224
# 비디오에서 학습할 프레임 개수
MAX_SEQ_LENGTH = 20
BATCH_SIZE = 32

# 데이터 경로
data_path = '/content/drive/MyDrive/workspace/4th_project/dataset/'

In [ ]:

train_df = pd.read_csv(data_path + 'train.csv')
test_df = pd.read_csv(data_path + 'test.csv')

train_df["label"] = train_df["tag"]
test_df["label"] = test_df["tag"]

for index, data in enumerate(train_df["label"].unique()):
    # labe 컴럼에 저장된 data를 index로 변환
    train_df["label"].replace(data, index, inplace=True)

for index, data in enumerate(test_df["label"].unique()):
    # labe 컴럼에 저장된 data를 index로 변환
    test_df["label"].replace(data, index, inplace=True)    

# sklearn.utils.shuffle(train_df) : train_df 행을 섞음
train_df = sklearn.utils.shuffle(train_df)

In [ ]:
# 비디오를 구성하는 이미지의 가운데 부분을 리턴
# frame : 비디오를 구성하는 이미지
def crop_center_square(frame):
    # frame.shape[0:2] : 이미지의 세로, 가로 리턴
    y, x = frame.shape[0:2]
    # 이미지의 세로 가로 중에서 작은 값을 리턴
    min_dim = min(y, x)
    #이미지의 왼쪽 모서리 좌표의 가로 시작점
    # 이미지 가로 좌표 (이미지 가로 //2  - min_dim//2) ~ (이미지 가로 //2  + min_dim//2) 
    start_x = (x // 2) - (min_dim // 2)
    # 이미지의 왼쪽 모서리 세로 좌표 시작점
    # 이미지 세로 좌표 (이미지 세로 //2  - min_dim//2) ~ (이미지 세로 //2  + min_dim//2) 
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


# 비디오 파일을 읽어서 각 프레임을 이미지로 변환해서 리턴
# path : 비디오 파일 경로
# max_frames : 이미지로 변환할 프레임수
# resize=(IMG_SIZE, IMG_SIZE) : 이미지의 가로 세로
def load_video(path, max_frames=20, resize=(IMG_SIZE, IMG_SIZE)):
    # 비디오 파일을 읽어서 이미지로 변환 할 객페
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            # cap.read() : 비디오를 읽어서 리턴
            # ret : 비디오 읽기가 성공했으면 True, 더이상 읽을 비디오 프레임이 없으면 False 가 리턴
            # frame : 비디오 프레임 이미지를 리턴
            ret, frame = cap.read()
            # ret 가 False면 반복 종료
            if not ret:
                break
            # 비디오 이미지를 가운데 리턴
            frame = crop_center_square(frame)
            #비디오 이미지의 가로 세로를 resize=(224,224) 로 변환
            frame = cv2.resize(frame, resize)
            # frame 은  [줄, 칸, B G R ] 로 구성되 있음 
            # frame [ : (모든줄), : (모든칸), R (인덱스2) G (인덱스1) B (인덱스0 )] 리턴
            frame = frame[:, :, [2, 1, 0]]
            # frame을 frames에 추가
            frames.append(frame)
            # frames에 저장된 데이터수가 max_frames와 같으면 종료
            if len(frames) == max_frames:
                break
    finally:
        # 비디오 이미지 변환 종료
        cap.release()
    # frames를 numpy 배열로 변환 해서 리턴
    return np.array(frames)

In [ ]:
# 모든 비디오 파일의 이미지와 종류를 리턴
# video_name : 비디오 파일명
# label : 비디오 파일 종류
# root_dir : 비디오 파일 경로
def prepare_all_videos(video_name, label, root_dir):
    # 비디오 파일 이름의 개수
    num_samples = len(video_name)
    # video_name.values.tolist() : 비디오 파일 이름을 리스트로 변환해서 리턴
    video_paths = video_name.values.tolist()
    # 비디오 종류를 리턴
    labels = label.values
    # 비디오 종류를 2차원 배열로 변환
    labels = labels.reshape(-1,1)
    # 0으로 초기화된 [비디오 파일개수 * 20, 224, 224, 3] 배열 생성
    x = np.zeros(shape=(num_samples * MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3), dtype="float32")
    # 0으로 초기화된 [비디오 파일개수 * 20]인 배열 생성
    y = np.zeros(shape=(num_samples * MAX_SEQ_LENGTH), dtype="float32")

    index = 0
    # 비디오 파일의 개수 만큼 반복
    for idx, path in enumerate(video_paths):
        # 비디오 파일을 읽어서 각 프레임의 이미지를 리턴
        frames = load_video(root_dir + path)
        # 프레임의 개수만큼 반복
        for i in range(len(frames)):
            # frame의 i번째 이미지를 x에 추가
            x[index] = np.array(frames[i], dtype="float32")
            # 비디오의 종류가 저장된 labels의 idx 번째를 y에 추가
            y[index] = np.array(labels[idx], dtype="float32")
            # index 1증가
            index += 1

    # 전체 비디오 프레임을 이미지로 변환한 x
    # to_categorical(y) : 이미지의 종류 y를 onehot 인코딩
    return (x, to_categorical(y))

In [ ]:
X_train, y_train = prepare_all_videos(train_df["video_name"], train_df["label"], data_path + "train/")
X_test, y_test = prepare_all_videos(test_df["video_name"], test_df["label"], data_path + "test/")

# LeNet5 모델

In [ ]:
# 이미지 데이터 생성
train_image_data_generator = ImageDataGenerator(
	horizontal_flip=True,
    rescale=1/255
)

test_image_data_generator = ImageDataGenerator(
    rescale=1/255
)

In [ ]:
# callbacks

# 학습하여 나온 결과가 개선되지 않으면 학습 중단
early_stopping = EarlyStopping(
    monitor='loss',               # 무엇을 감시하고 있을지
    min_delta=1e-4,               # 개선이 되는 것으로 보는 최소값, 이 값보다 작으면 개선이 없는 것
    patience=10,                  # patience의 epochs만큼 진행해도 개선되지 않으면 중단
    verbose=1,                    # 화면 출력 관련
    # mode='min'                  # 여기서는 min이 필요하지만 auto로 알아서 해줌
    # baseline=0.01               # 이것보다 개선되지 않으면 학습 중단
    # restore_best_weights=False  # True: 가장 높게 나온 모델, False: 마지막 학습 모델
)

# 모델 또는 가중치를 저장하기 위한 콜백이나 사용하지 않을 예정입니다. (참고삼아 넣었습니다.)
model_checkpoint = ModelCheckpoint(
    filepath='{가중치를 저장할 경로 입력}',
    monitor='loss'
)

## model_1

In [ ]:
num_classes = 3
class LeNet_1(Sequential):
  def __init__(self, input_shape, nb_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = "relu", input_shape = input_shape, padding = "same"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = "relu", padding = "valid"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Flatten())
    self.add(Dense(120, activation = "relu"))
    self.add(Dense(84, activation = "relu"))
    self.add(Dense(nb_classes, activation = "softmax"))

    self.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model_1 = LeNet_1((IMG_SIZE, IMG_SIZE, 3), num_classes)
model_1.summary()

Model: "le_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 6)       456       
_________________________________________________________________
average_pooling2d (AveragePo (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 16)      2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 54, 54, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 46656)             0         
_________________________________________________________________
dense (Dense)                (None, 120)               5598840   
_________________________________________________________________
dense_1 (Dense)              (None, 84)                101

In [ ]:
model_1.fit(
    train_image_data_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=100,
    callbacks=[early_stopping],
    steps_per_epoch=len(X_train) // BATCH_SIZE,
)

Epoch 1/100
222/222 [==============================] - 21s 23ms/step - loss: 0.0822 - accuracy: 0.9680
Epoch 2/100
222/222 [==============================] - 5s 23ms/step - loss: 0.0244 - accuracy: 0.9941
Epoch 3/100
222/222 [==============================] - 5s 23ms/step - loss: 2.9001e-04 - accuracy: 1.0000
Epoch 4/100
222/222 [==============================] - 5s 23ms/step - loss: 5.7544e-05 - accuracy: 1.0000
Epoch 5/100
222/222 [==============================] - 5s 23ms/step - loss: 3.0738e-05 - accuracy: 1.0000
Epoch 6/100
222/222 [==============================] - 5s 23ms/step - loss: 1.6961e-05 - accuracy: 1.0000
Epoch 7/100
222/222 [==============================] - 5s 23ms/step - loss: 1.1251e-05 - accuracy: 1.0000
Epoch 8/100
222/222 [==============================] - 5s 23ms/step - loss: 7.6540e-06 - accuracy: 1.0000
Epoch 9/100
222/222 [==============================] - 5s 23ms/step - loss: 5.5743e-06 - accuracy: 1.0000
Epoch 10/100
222/222 [==============================]

In [ ]:
model_1.evaluate(
    test_image_data_generator.flow(X_test, y_test, batch_size=BATCH_SIZE)
)

86/86 [==============================] - 2s 20ms/step - loss: 3.2110 - accuracy: 0.7296


[3.210982322692871, 0.7295620441436768]

## model_2

In [ ]:
class LeNet_2(Sequential):
  def __init__(self, input_shape, nb_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = "relu", input_shape = input_shape, padding = "same"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = "relu", padding = "valid"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Flatten())
    self.add(Dense(120, activation = "relu"))
    self.add(Dense(84, activation = "relu"))
    self.add(Dense(nb_classes, activation = "softmax"))

    self.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model_2 = LeNet_2((IMG_SIZE, IMG_SIZE, 3), num_classes)
model_2.summary()

Model: "le_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
average_pooling2d_2 (Average (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 108, 108, 16)      2416      
_________________________________________________________________
average_pooling2d_3 (Average (None, 54, 54, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 46656)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               5598840   
_________________________________________________________________
dense_4 (Dense)              (None, 84)                101

In [ ]:
model_2.fit(
    train_image_data_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=100,
    callbacks=[early_stopping],
    steps_per_epoch=len(X_train) // BATCH_SIZE,
)

Epoch 1/100
222/222 [==============================] - 6s 24ms/step - loss: 0.1667 - accuracy: 0.9475
Epoch 2/100
222/222 [==============================] - 5s 24ms/step - loss: 0.0537 - accuracy: 0.9859
Epoch 3/100
222/222 [==============================] - 5s 24ms/step - loss: 5.6694e-04 - accuracy: 1.0000
Epoch 4/100
222/222 [==============================] - 5s 24ms/step - loss: 9.0577e-05 - accuracy: 1.0000
Epoch 5/100
222/222 [==============================] - 5s 24ms/step - loss: 6.1483e-05 - accuracy: 1.0000
Epoch 6/100
222/222 [==============================] - 5s 24ms/step - loss: 3.2016e-05 - accuracy: 1.0000
Epoch 7/100
222/222 [==============================] - 5s 24ms/step - loss: 2.2085e-05 - accuracy: 1.0000
Epoch 8/100
222/222 [==============================] - 5s 24ms/step - loss: 1.5724e-05 - accuracy: 1.0000
Epoch 9/100
222/222 [==============================] - 5s 22ms/step - loss: 1.1412e-05 - accuracy: 1.0000
Epoch 10/100
222/222 [==============================] 

In [ ]:
model_2.evaluate(
    test_image_data_generator.flow(X_test, y_test, batch_size=BATCH_SIZE)
)

86/86 [==============================] - 2s 19ms/step - loss: 3.3167 - accuracy: 0.6248


[3.3167078495025635, 0.6248174905776978]

## model_3

In [ ]:
class LeNet_3(Sequential):
  def __init__(self, input_shape, nb_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = "relu", input_shape = input_shape, padding = "same"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = "relu", padding = "valid"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Flatten())
    self.add(Dense(120, activation = "relu"))
    self.add(Dense(84, activation = "relu"))
    self.add(Dense(nb_classes, activation = "softmax"))

    self.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model_3 = LeNet_3((IMG_SIZE, IMG_SIZE, 3), num_classes)
model_3.summary()

Model: "le_net_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
average_pooling2d_4 (Average (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 108, 108, 16)      2416      
_________________________________________________________________
average_pooling2d_5 (Average (None, 54, 54, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 46656)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 120)               5598840   
_________________________________________________________________
dense_7 (Dense)              (None, 84)                101

In [ ]:
model_3.fit(
    train_image_data_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=100,
    callbacks=[early_stopping],
    steps_per_epoch=len(X_train) // BATCH_SIZE,
)

Epoch 1/100
222/222 [==============================] - 5s 22ms/step - loss: 0.0771 - accuracy: 0.9707
Epoch 2/100
222/222 [==============================] - 5s 22ms/step - loss: 0.0015 - accuracy: 0.9994
Epoch 3/100
222/222 [==============================] - 5s 22ms/step - loss: 4.3412e-05 - accuracy: 1.0000
Epoch 4/100
222/222 [==============================] - 5s 22ms/step - loss: 1.6417e-05 - accuracy: 1.0000
Epoch 5/100
222/222 [==============================] - 5s 22ms/step - loss: 0.0511 - accuracy: 0.9848
Epoch 6/100
222/222 [==============================] - 5s 22ms/step - loss: 0.0086 - accuracy: 0.9976
Epoch 7/100
222/222 [==============================] - 5s 23ms/step - loss: 0.0025 - accuracy: 0.9989
Epoch 8/100
222/222 [==============================] - 5s 22ms/step - loss: 9.4500e-06 - accuracy: 1.0000
Epoch 9/100
222/222 [==============================] - 5s 22ms/step - loss: 6.1293e-06 - accuracy: 1.0000
Epoch 10/100
222/222 [==============================] - 5s 22ms/st

In [ ]:
model_3.evaluate(
    test_image_data_generator.flow(X_test, y_test, batch_size=BATCH_SIZE)
)

86/86 [==============================] - 2s 18ms/step - loss: 1.5466 - accuracy: 0.7175


[1.5465978384017944, 0.7175182700157166]

## model_4

In [ ]:
class LeNet_4(Sequential):
  def __init__(self, input_shape, nb_classes):
    super().__init__()

    self.add(Conv2D(6, kernel_size = (5, 5), strides = (1, 1), activation = "relu", input_shape = input_shape, padding = "same"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Conv2D(16, kernel_size = (5, 5), strides = (1, 1), activation = "relu", padding = "valid"))
    self.add(AveragePooling2D(pool_size = (2, 2), strides = (2, 2), padding = "valid"))

    self.add(Flatten())
    self.add(Dense(120, activation = "relu"))
    self.add(Dense(84, activation = "relu"))
    self.add(Dense(nb_classes, activation = "softmax"))

    self.compile(optimizer = "adam", loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model_4 = LeNet_4((IMG_SIZE, IMG_SIZE, 3), num_classes)
model_4.summary()

Model: "le_net_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 224, 224, 6)       456       
_________________________________________________________________
average_pooling2d_6 (Average (None, 112, 112, 6)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 108, 108, 16)      2416      
_________________________________________________________________
average_pooling2d_7 (Average (None, 54, 54, 16)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 46656)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 120)               5598840   
_________________________________________________________________
dense_10 (Dense)             (None, 84)                101

In [ ]:
model_4.fit(
    train_image_data_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=100,
    callbacks=[early_stopping],
    steps_per_epoch=len(X_train) // BATCH_SIZE,
)

Epoch 1/100
222/222 [==============================] - 6s 23ms/step - loss: 0.1211 - accuracy: 0.9588
Epoch 2/100
222/222 [==============================] - 5s 23ms/step - loss: 0.0062 - accuracy: 0.9986
Epoch 3/100
222/222 [==============================] - 5s 23ms/step - loss: 2.0661e-04 - accuracy: 1.0000
Epoch 4/100
222/222 [==============================] - 5s 23ms/step - loss: 4.8292e-05 - accuracy: 1.0000
Epoch 5/100
222/222 [==============================] - 5s 23ms/step - loss: 2.4398e-05 - accuracy: 1.0000
Epoch 6/100
222/222 [==============================] - 5s 23ms/step - loss: 1.4810e-05 - accuracy: 1.0000
Epoch 7/100
222/222 [==============================] - 5s 23ms/step - loss: 1.0891e-05 - accuracy: 1.0000
Epoch 8/100
222/222 [==============================] - 5s 23ms/step - loss: 7.8095e-06 - accuracy: 1.0000
Epoch 9/100
222/222 [==============================] - 5s 22ms/step - loss: 6.2063e-06 - accuracy: 1.0000
Epoch 10/100
222/222 [==============================] 

In [ ]:
model_4.evaluate(
    test_image_data_generator.flow(X_test, y_test, batch_size=BATCH_SIZE)
)

86/86 [==============================] - 2s 19ms/step - loss: 0.8559 - accuracy: 0.8588


[0.8558628559112549, 0.8587591052055359]

# 앙상블

In [ ]:
model_1.save('/content/drive/MyDrive/workspace/4th_project/models/LeNet5_ESB_01.h5')
model_2.save('/content/drive/MyDrive/workspace/4th_project/models/LeNet5_ESB_02.h5')
model_3.save('/content/drive/MyDrive/workspace/4th_project/models/LeNet5_ESB_03.h5')
model_4.save('/content/drive/MyDrive/workspace/4th_project/models/LeNet5_ESB_04.h5')

In [ ]:
preds_a = model_1.predict(X_test)
preds_b = model_2.predict(X_test)
preds_c = model_3.predict(X_test)
preds_d = model_4.predict(X_test)

## Normal Ensemble

In [ ]:
final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)

In [ ]:
predict = np.where( final_preds > 0.5, 1,0)
predict = np.array(predict, dtype="float32")
y_test = np.array(y_test, dtype="float32")
predict2 = (predict == y_test)
acc = 1- (len(y_test)*3 - np.sum(predict2)) / len(y_test)
acc

0.6248175182481752

## Nelder_mead

In [ ]:
final_preds_2 = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d

In [ ]:
predict = np.where( final_preds_2 > 0.5, 1,0)
predict = np.array(predict, dtype="float32")
y_test = np.array(y_test, dtype="float32")
predict2 = (predict == y_test)
acc = 1- (len(y_test)*3 - np.sum(predict2)) / len(y_test)
acc

0.5470802919708029

## Best model

In [ ]:
model_4.evaluate(
    test_image_data_generator.flow(X_test, y_test, batch_size=BATCH_SIZE)
)

86/86 [==============================] - 2s 19ms/step - loss: 0.8559 - accuracy: 0.8588


[0.8558628559112549, 0.8587591052055359]